## MLB EDA and Modeling Prep
### Explore MLB datasets loaded by `mlb_stats_etl`, and prepare modeling-ready features to predict outcomes such as:
### - Home team win/loss
### - Total score
### - Player RBIs for an upcoming game

In [ ]:
# Imports and config
import os
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sqlalchemy import create_engine
from dotenv import load_dotenv

sns.set_theme(style="whitegrid")
pd.set_option("display.max_columns", 200)

# Load .env from project root if present
ROOT = Path(__file__).resolve().parents[1]
load_dotenv(ROOT / ".env")

# Toggle sources
USE_DB = True  # set False to load from Parquet files in PARQUET_DIR
PARQUET_DIR = ROOT / "out_full"  # adjust as needed if you wrote elsewhere

DB_URL = os.getenv("DATABASE_URL") or (
    f"mysql+pymysql://{os.getenv('DB_USER')}:{os.getenv('DB_PASSWORD')}@"
    f"{os.getenv('DB_HOST')}:{os.getenv('DB_PORT')}/{os.getenv('DB_DATABASE')}?charset=utf8mb4"
)
ENGINE = create_engine(DB_URL) if USE_DB else None
print("DB_URL:", DB_URL if USE_DB else "<parquet mode>")

### Load core tables
### Adjust SEASON_START and SEASON_END to scope the data volume.

In [ ]:

SEASON_START = 2015
SEASON_END = 2025


def read_table(name: str) -> pd.DataFrame:
    if USE_DB:
        q = f"SELECT * FROM `{name}`"
        # season filter for large tables
        if name in {"games", "game_players", "game_teams", "linescores", "plays", "pitches"}:
            q += f" WHERE season BETWEEN {SEASON_START} AND {SEASON_END}"
        return pd.read_sql(q, ENGINE)
    p = PARQUET_DIR / f"{name}.parquet"
    return pd.read_parquet(p) if p.exists() else pd.DataFrame()


teams = read_table("teams")
games = read_table("games")
game_teams = read_table("game_teams")
game_players = read_table("game_players")
linescores = read_table("linescores")
standings = read_table("standings")
print("Shapes:", games.shape, game_teams.shape, game_players.shape, linescores.shape)